# Clustering

The goal of this program is to visualize and cluster the data.

## Input
* Importing the libaries that are necessary:

In [1]:
import numpy as np
import pandas as pd

* Importing the necessary files:
 * Read files
 * Convert read files to data
 * Save data in dataframes

In [2]:
df_data = pd.read_csv('Data\Voorbeeld_clusterdata.txt', sep='  ', header=None, engine='python', index_col=0)
df_results = pd.read_csv('Data\Voorbeeld_clusterresult.txt', sep='  ', header=None, engine='python', index_col=0)
df_clusterd = df_data.copy().drop(columns=df_data.columns)

df_data.head(10)

,1,2,3,4,5,6,7,8
0,,,,,,,,
846160,0.388,0.329,0.690,0.900,0.626,0.621,0.399,0.370
820434,-0.296,-0.503,-0.454,-0.868,-0.721,-0.918,-0.486,-0.582
849103,-0.246,-0.935,-0.277,-0.175,-0.278,-0.075,-0.236,-0.417
846353,0.769,0.929,0.977,1.095,1.058,0.864,0.689,0.492
848613,-0.365,-0.760,-0.305,-0.293,-0.364,-0.155,-0.472,-0.606
846978,0.858,1.134,0.950,1.806,1.384,1.414,1.169,1.146
847332,-0.029,-0.018,-0.285,-0.813,-0.974,-0.936,-0.607,-0.514
847300,0.337,0.067,-0.762,-2.677,-2.772,-3.670,-1.356,-1.443
846924,0.552,0.743,0.581,1.027,0.754,0.596,0.712,0.506


## Cluster methods
### K-means (KMCA):

uitleggen hoe het werkt

In [3]:
class KMCA:
    def __init__(self, k=6, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        
    def set_k(K):
        self.k = K

    def fit(self,data):

        self.centroids = {}

        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    optimized = False

            if optimized:
                break

    def predict(self,data):
        def pred(self, data):
            distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
            classification = distances.index(min(distances))
            return classification
        
        self.pred_classifications = list()
        
        if np.isfortran(data):
            for element in data:
                self.pred_classifications.append(pred(self, element))
        else:
            self.pred_classifications.append(pred(self, data))
            
        return self.pred_classifications

### Own cluster method:

## Output

* get cluster labels
* save labels in text file

### get cluster labels:

In [4]:
kmca = KMCA()
kmca.fit(df_data.values)
df_clusterd['KMCA'] = kmca.predict(df_data.values)

df_clusterd.head()

,KMCA
0,
846160,0
820434,4
849103,2
846353,3
848613,2


### save results:

In [12]:
df_clusterd.to_csv(r'Data\clusterresultaten.txt', header=None, index=True, sep=' ')